In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature = 0.1
)

cache_dir = LocalFileStore("../.cache/")

loader = UnstructuredFileLoader("../files/chapter_one.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(  
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
    ),
    ("human", "{question}"),
]) 

chain = {"context": retriver, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Vicory Mansions.")


AIMessage(content="Victory Mansions is a building where Winston Smith resides. It has glass doors at the entrance and is located in a grimy landscape. The hallway of Victory Mansions has a smell of boiled cabbage and old rag mats. There is a large coloured poster on one end of the hallway, depicting the face of a man with a black moustache. The building has seven floors, and Winston's flat is on the seventh floor. The flat is described as small and has a window overlooking the outside. The building is not well-maintained, and the elevator is often out of service due to the electricity being cut off during daylight hours.")